In [4]:
import pandas as pd

In [5]:
df_17 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2017.csv', encoding='utf-16')
df_18 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2018.csv', encoding='utf-16')
df_19 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2019.csv', encoding='utf-16')
df_20 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2020.csv', encoding='utf-16')
df_21 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2021.csv', encoding='utf-16')
df_22 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2022.csv', encoding='utf-16')
df_23 = pd.read_csv('/Users/xuanedx1/github/outage-data-scraper/data/bluefire/POUS_Export_CityByUtility_Raw_2023.csv', encoding='utf-16')


In [6]:
df_23

,UtilityName,StateName,CountyName,CityName,CountyFIPS,CustomersTracked,CustomersOut,RecordDateTime
0,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,982,0,2023-03-17 16:48:34
1,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,983,0,2023-03-29 05:28:08
2,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,982,0,2023-05-24 05:48:41
3,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,983,0,2023-06-06 05:05:54
4,4 Rivers Electric Coop,Kansas,Anderson,Unknown,20003.0,981,0,2023-06-22 05:55:10
...,...,...,...,...,...,...,...,...
12077308,Y-W Electric Association,Nebraska,Dundy,Sanborn,31057.0,67,0,2023-10-25 21:25:03
12077309,Y-W Electric Association,Nebraska,Dundy,Sanborn,31057.0,67,1,2023-10-30 21:39:07
12077310,Y-W Electric Association,Nebraska,Dundy,Sanborn,31057.0,67,0,2023-10-30 22:01:36
12077311,Y-W Electric Association,Nebraska,Dundy,Sanborn,31057.0,67,2,2023-11-06 17:12:50


In [7]:
def transform(df):
    # Ensure RecordDateTime is a datetime type
    df['RecordDateTime'] = pd.to_datetime(df['RecordDateTime'])
    
    # Extract and create year, month, day, hour, and date columns
    df['Year'] = df['RecordDateTime'].dt.year
    df['Month'] = df['RecordDateTime'].dt.month
    df['Day'] = df['RecordDateTime'].dt.day
    df['Hour'] = df['RecordDateTime'].dt.hour
    df['Date'] = df['RecordDateTime'].dt.date  # Date without time component
    
    return df

def aggregate(df):
    aggregation_rules = {
        'CustomersTracked': 'max',
        'CustomersOut': 'max',
    }
    
    # Group by the specified columns plus the datetime components and aggregate
    aggregated_df = df.groupby(['UtilityName', 'StateName', 'CountyName', 'CountyFIPS', 'Date', 'Year', 'Month', 'Day', 'Hour'], as_index=False).agg(aggregation_rules)
    
    return aggregated_df

In [8]:
years=[2017, 2018, 2019, 2020, 2021, 2022, 2023]
for year,df in [[2017, df_17], [2018, df_18], [2019, df_19], [2020, df_20], [2021, df_21], [2022, df_22], [2023, df_23]]:
    df_t = transform(df)
    df_agg = aggregate(df_t)
    df_agg.to_csv(f'/Users/xuanedx1/github/outage-data-scraper/data/bluefire/by_hour_{year}.csv')

---

In [19]:
ga = df_23.query('StateName == "Georgia"')

In [24]:
import altair as alt
import pandas as pd

# Disable the max rows limit
alt.data_transformers.disable_max_rows()
# Assuming df is your DataFrame and already loaded

# Convert RecordDateTime to datetime if not already
ga['RecordDateTime'] = pd.to_datetime(ga['RecordDateTime'].copy())

# Now create the chart
chart = alt.Chart(ga).mark_line().encode(
    x='RecordDateTime:T',  # T indicates temporal (time) data
    y='CustomersOut:Q',    # Q indicates quantitative data
    color='CountyName:N'   # N indicates nominal (categorical) data
    # tooltip=['UtilityName:N', 'StateName:N', 'CountyName:N', 'CustomersOut:Q', 'RecordDateTime:T']
).properties(
    width=800,
    height=400,
    title='Customers Out Over Time by State'
).interactive()

chart.display()

/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_1105/1150667029.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ga['RecordDateTime'] = pd.to_datetime(ga['RecordDateTime'].copy())


TypeError: Object of type date is not JSON serializable

alt.Chart(...)